# Create Data

In [2]:
!python main.py --help

usage: main.py [-h] [-f F] [-t T] [-n N] [-p P] [-c C] [-k K]
               [--absolute_freqs] [--z_scores] [-s S [S ...]] [-x X]
               [--sampling] [--sample_units SAMPLE_UNITS]
               [--sample_size SAMPLE_SIZE] [--sample_step SAMPLE_STEP]
               [--max_samples MAX_SAMPLES] [--keep_punct] [--keep_sym]
               [--identify_lang]

options:
  -h, --help            show this help message and exit
  -f F                  optional list of features in json
  -t T                  types of features (words or chars)
  -n N                  n grams lengths (default 1)
  -p P                  Processes to use (default 1)
  -c C                  Path to file with metadata corrections
  -k K                  How many most frequent?
  --absolute_freqs      switch to get absolute instead of relative freqs
  --z_scores            Use z-scores?
  -s S [S ...]          paths to files
  -x X                  format (txt, xml or tei)
  --sampling            Sample the tex

In [3]:
# Different options for data
# first, character n-grams
!python main.py -s data/trouveres/train_cleaned_segm/*.txt -t chars -n 3 -x txt #--sampling --sample_units words --sample_size 3000

.......loading texts.......
.......getting features.......
K Limit ignored because the size of the list is lower (2490 < 5000)
.......getting counts.......
.......feeding data frame.......
100%|█████████████████████████████████████████| 129/129 [00:04<00:00, 32.09it/s]
.......applying normalisations.......
.......saving results.......


In [ ]:
# Or, affixes
!python main.py -s data/trouveres/train_cleaned_segm/*.txt -t chars -n 3 -x txt #--sampling --sample_units words --sample_size 3000
!python features_filter.py --affixes_grams -f feature_list_chars3grams5000mf.json
!python main.py -s data/trouveres/train_cleaned_segm/*.txt -t chars -n 3 -x txt -f feature_list_chars3grams5000mf_affixes.json #--sampling --sample_units words --sample_size 3000 

In [ ]:
# Words
!python main.py -s data/trouveres/train_cleaned_segm/*.txt -t words -n 1 -x txt #--sampling --sample_units words --sample_size 3000

In [ ]:
# Function words
!python main.py -s data/trouveres/train_cleaned_segm/*.txt -t words -n 1 -x txt -f function_words.json #--sampling --sample_units words --sample_size 3000 

In [ ]:
# Lemmas
!python main.py -s data/lem/train/*.txt -t words -n 1 -x txt #--sampling --sample_units words --sample_size 3000

In [ ]:
# Function lemmas
!python main.py -s data/txt/train/*.txt -t words -n 1 -x txt -f function_lem.json # --sampling --sample_units words --sample_size 3000 

# Analyse data

In [4]:
!python train_svm.py --help

usage: train_svm.py [-h] [--test_path TEST_PATH]
                    [--cross_validate {leave-one-out,k-fold,group-k-fold}]
                    [--k K] [--dim_reduc {pca}] [--norms]
                    [--balance {downsampling,Tomek,upsampling,SMOTE,SMOTETomek}]
                    [--class_weights]
                    [--kernel {LinearSVC,linear,polynomial,rbf,sigmoid}]
                    [--final] [--get_coefs]
                    train_path

positional arguments:
  train_path            Path to train file

options:
  -h, --help            show this help message and exit
  --test_path TEST_PATH
                        Path to test file
  --cross_validate {leave-one-out,k-fold,group-k-fold}
                        perform cross validation (test_path will be used only
                        for final prediction).If group_k-fold is chosen, each
                        source file will be considered a group (only relevant
                        if sampling was performed and more than 

In [6]:
!python train_svm.py feats_tests_n3_k_5000.csv --cross_validate leave-one-out --class_weights --get_coefs

.......... loading data ........
.......... Formatting data ........
.......... Creating pipeline according to user choices ........
.......... using normalisations ........
.......... choosing SVM ........
.......... Creating pipeline with steps ........
[('scaler', StandardScaler()), ('normalizer', Normalizer()), ('model', LinearSVC(class_weight='balanced'))]
.......... leave-one-out cross validation will be performed ........
.......... using 129 samples or groups........
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 129 out of 129 | elapsed:  1.1min finished
              precision    recall  f1-score   support

       Blond       0.64      0.43      0.51        21
        Gace       0.69      0.88      0.78        43
        Gaut       0.71      0.60      0.65        20
        Thib       0.98      0.93      0.95        45

    accuracy                           0.78       129
   macro avg       0.75      0.71      0.